# Importing Data

## Set up cell magic/games dataset

In [1]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [77]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

games = pd.read_csv(r"/Users/shobhanashreedhar/SteamRecommender/games_may2024_cleaned.csv")

In [14]:
rev_small = pd.read_csv('rev_small.csv', dtype=dtypedict)

In [8]:
dtypedict = {'Unnamed: 0': 'Int64', 'recommendationid': 'Int64', 'language': 'object', 'timestamp_created': 'Int64', 'timestamp_updated': 'Int64',
            'voted_up': 'Int64', 'votes_up': 'Int64', 'votes_funny': 'Int64', 'weighted_vote_score': 'float', 'comment_count': 'Int64',
             'steam_purchase': 'Int64', 'received_for_free': 'Int64', 'written_during_early_access': 'Int64', 'hidden_in_steam_china': 'Int64',
             'steam_china_location': 'object', 'author_steamid': 'Int64','author_num_games_owned': 'Int64', 'author_num_reviews': 'Int64', 
             'author_playtime_forever': 'float', 'author_playtime_last_two_weeks': 'float', 'author_playtime_at_review': 'float', 
             'author_last_played': 'Int64', 'AppID': 'Int64', 'Unnamed: 0.1': 'Int64'}

## Joining all reviews datasets

In [3]:
%%skip
import os

reviewdir = '/Users/shobhanashreedhar/SteamReviews2024'

review_files = {}

In [4]:
%%skip
for f in os.listdir(reviewdir):
    fpath = os.path.join(reviewdir, f)
    key = int(f[:-4])
    review_files[key] = fpath
    df = pd.read_csv(fpath)
    df['AppID'] = key
    df.to_csv(fpath)
    print('appended AppID to', fpath)

In [5]:
%%skip
games = games[games['AppID'].isin(review_files.keys())]

In [6]:
%%skip
import dask.dataframe as dd

reviewfiles = '/Users/shobhanashreedhar/SteamReviews2024/*.csv'
ddf = dd.read_csv(reviewfiles, assume_missing=True, dtype={'steam_china_location': 'object'})
reviews = ddf.compute()
reviews.to_csv('reviews.csv', index=False)


## Set up datasets

In [9]:
%%skip
reviews = pd.read_csv(r"C:\Users\mckid\Downloads\reviews.csv", dtype=dtypedict)

In [11]:
%%skip
reviews

In [12]:
%%skip
rev_small = reviews.sample(n=100000)
rev_small.to_csv('rev_small.csv')

In [13]:
%%skip
del reviews

# Data Wrangling

## Dropping Irrelevant Columns From Base Datasets

In [16]:
rev_small.drop(['Unnamed: 0.2', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [18]:
print('Null Percentages in rev_small:\n------------------------------')
for col in rev_small.columns.to_list():
    print(f"{col:<30} {(rev_small[col].isna().sum()/len(rev_small)):>20.3%}")

Null Percentages in rev_small:
------------------------------
recommendationid                             0.000%
language                                     0.000%
timestamp_created                            0.000%
timestamp_updated                            0.000%
voted_up                                     0.000%
votes_up                                     0.000%
votes_funny                                  0.000%
weighted_vote_score                          0.000%
comment_count                                0.000%
steam_purchase                               0.000%
received_for_free                            0.000%
written_during_early_access                  0.000%
hidden_in_steam_china                        0.000%
steam_china_location                        99.997%
author_steamid                               0.000%
author_num_games_owned                       0.000%
author_num_reviews                           0.000%
author_playtime_forever                      0.000%
au

We will drop the column `steam_china_location` due to its high null percentage (>50%) and lack of relevance to the dataset.

In [19]:
rev_small.drop('steam_china_location', axis=1, inplace=True)

In [20]:
print('Null Percentages in games:\n--------------------------')

for col in games.columns.to_list():
    print(f"{col:<30} {(games[col].isna().sum()/len(games)):>20.3%}")

Null Percentages in games:
--------------------------
AppID                                        0.000%
name                                         0.004%
release_date                                 0.000%
required_age                                 0.000%
price                                        0.000%
dlc_count                                    0.000%
detailed_description                         0.198%
about_the_game                               0.225%
short_description                            0.130%
reviews                                     87.707%
header_image                                 0.000%
website                                     52.466%
support_url                                 49.692%
support_email                               12.647%
windows                                      0.000%
mac                                          0.000%
linux                                        0.000%
metacritic_score                             0.000%
metacritic

We will drop the columns `reviews`, `website`, `support_url`, `metacritic_url`, `notes`, and `score_rank` due to high null percentages (>50%) and irrelevance to the dataset.

In [21]:
games.drop(['reviews', 'website', 'support_url', 'metacritic_url', 'notes', 'score_rank'], axis=1, inplace=True)

## Processing Combined Dataset

In [67]:
combined = pd.merge(games, rev_small, on='AppID', how='inner')

In [68]:
dropcandidates = ['required_age', 'dlc_count', 'metacritic_score', 'user_score', 'average_playtime_forever', 'average_playtime_2weeks',
                  'median_playtime_forever', 'median_playtime_2weeks', 'peak_ccu', 'votes_up', 'votes_funny', 'weighted_vote_score',
                  'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access', 'hidden_in_steam_china',
                  'author_playtime_forever', 'author_playtime_last_two_weeks', 'author_playtime_at_review', 'author_last_played',
                  'achievements']

In [69]:
droplist = []
print('Percentage of Values == 0 per Column:\n-------------------------------------')
for col in dropcandidates:
    zero_pct = (combined[col] == 0).sum() / len(combined)
    if zero_pct > 0.5:
        droplist.append(col)
    print(f"{col:<35} {zero_pct:>20.3%}")

Percentage of Values == 0 per Column:
-------------------------------------
required_age                                     77.048%
dlc_count                                        32.879%
metacritic_score                                 50.354%
user_score                                       99.990%
average_playtime_forever                         34.175%
average_playtime_2weeks                          52.065%
median_playtime_forever                          34.175%
median_playtime_2weeks                           52.065%
peak_ccu                                          5.765%
votes_up                                         70.071%
votes_funny                                      88.462%
weighted_vote_score                              66.339%
comment_count                                    96.056%
steam_purchase                                   34.692%
received_for_free                                95.878%
written_during_early_access                      88.546%
hidden_in_st

We will drop columns with >50% values == 0.

In [70]:
combined.drop(droplist, axis=1, inplace=True)

In [73]:
combined.drop(['detailed_description', 'about_the_game', 'short_description', 'header_image', 'support_email', 'packages', 'screenshots',
               'timestamp_created', 'timestamp_updated', 'author_last_played', 'movies', 'tags', 'estimated_owners', 'num_reviews_recent', 
               'full_audio_languages', 'publishers'], axis=1, inplace=True)

## OneHot Encoding

We will OneHot encode the columns with multiple values to ensure that our data is useful to the model.

In [79]:
from sklearn.preprocessing import OneHotEncoder